In [1]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from collections import Counter
import math
from math import log

In [2]:

# 书上题目5.1
def create_data():
    datasets = [['青年', '否', '否', '一般', '否'],
               ['青年', '否', '否', '好', '否'],
               ['青年', '是', '否', '好', '是'],
               ['青年', '是', '是', '一般', '是'],
               ['青年', '否', '否', '一般', '否'],
               ['中年', '否', '否', '一般', '否'],
               ['中年', '否', '否', '好', '否'],
               ['中年', '是', '是', '好', '是'],
               ['中年', '否', '是', '非常好', '是'],
               ['中年', '否', '是', '非常好', '是'],
               ['老年', '否', '是', '非常好', '是'],
               ['老年', '否', '是', '好', '是'],
               ['老年', '是', '否', '好', '是'],
               ['老年', '是', '否', '非常好', '是'],
               ['老年', '否', '否', '一般', '否'],
               ]
    labels = [u'年龄', u'有工作', u'有自己的房子', u'信贷情况', u'类别']
    # 返回数据集和每个维度的名称
    return datasets, labels

In [3]:
datasets, labels = create_data()
train_data = pd.DataFrame(datasets, columns=labels)
train_data

,年龄,有工作,有自己的房子,信贷情况,类别
0,青年,否,否,一般,否
1,青年,否,否,好,否
2,青年,是,否,好,是
3,青年,是,是,一般,是
4,青年,否,否,一般,否
5,中年,否,否,一般,否
6,中年,否,否,好,否
7,中年,是,是,好,是
8,中年,否,是,非常好,是
9,中年,否,是,非常好,是


### 计算熵
熵(entropy)是表示随机变量不确定性的度量，设X是一个取有限个值的离散随机变量，其概率分布为:
$$P(X = x_{i}) = p_{i}, i = 1,2,...,n$$

则随机变量X的熵定义为:
$$H(X) = - \sum_{i=1}^{n}p_{i}logp_{i}$$

由定义可以看出熵只依赖于X的分布，而与X的取值无关，所以将X的熵记做H(p)
$$H(p) = - \
sum_{i=1}^{n}p_{i}logp_{i}$$

其中熵越大，随机变量的不确定性就越大

In [4]:
# 计算熵
def calc_ent(datasets):
    data_length = len(datasets)
    label_count = dict()
    for i in range(data_length):
        label = datasets[i][-1]
        label_count[label] = label_count.get(label,0) + 1
    ent = -sum([(p/data_length) * log(p/data_length,2) for p in label_count.values()])
    return ent

### 条件熵
设有随机变量(X,Y)，其联合概率分布为
$$P(X = x_{i},Y = y_{j}) = p_{ij}, i = 1,2,...,n;j = 1,2,...,m$$

条件熵$H(Y|X)$表示在已知随机变量X的条件下随机变量Y的不确定性，定义为X给定条件下Y的条件概率分布的熵对X的数学期望
$$H(Y|X) = \sum_{i=1}^{n}p_{i}H(Y|X = x_{i})$$
其中 $p_{i} = P(X = x_{i}), i = 1,2,...,n$

In [5]:
# 计算条件熵
def cond_ent(datasets,axis=0):
    # 计算条件熵肯定要借助于计算熵
    data_length = len(datasets)
    feature_sets = dict()
    for i in range(data_length):
        feature = datasets[i][axis]
        if feature not in feature_sets:
            feature_sets[feature] = []
        feature_sets[feature].append(datasets[i])
    cond_sum = sum([(len(p)/data_length) * calc_ent(p) for p in feature_sets.values()])
    return cond_sum

#### 信息增益
特征A对训练数据集D的信息增益g(D,A)，定义为集合D的经验熵H(D)与特征A给定条件下D的经验熵H(D|A)之差:
$$g(D,A) = H(D) - H(D|A)$$

In [24]:
# 信息增益
def info_gain(ent,cond_ent,label='minis'):
    if label == 'minis':
        return ent - cond_ent
    else:
        return cond_ent/ent
    
def info_gain_train(datasets):
    # datasets 最后一列是类别
    feature_num = len(datasets[0]) - 1
    ent = calc_ent(datasets)
    
    best_feature = []
    for c in range(feature_num):
        c_info_gain = info_gain(ent ,cond_ent(datasets,axis=c))
        best_feature.append((c,c_info_gain))
        print('特征({}) - info_gain - {:.3f}'.format(labels[c], c_info_gain))
    # 比较大小
    best_ = max(best_feature, key=lambda x: x[-1])
    return '特征({})的信息增益最大，选择为根节点特征'.format(labels[best_[0]])

In [9]:
info_gain_train(np.array(datasets))

特征(年龄) - info_gain - 0.083
特征(有工作) - info_gain - 0.324
特征(有自己的房子) - info_gain - 0.420
特征(信贷情况) - info_gain - 0.363


'特征(有自己的房子)的信息增益最大，选择为根节点特征'

#### 决策树（ID3）/C4.5

&emsp;&emsp;ID3算法的核心是在决策树各个节点上应用信息增益准则选择特征，递归的构建决策树。具体方法是：从根节点开始，对节点计算所有可能的信息增益，选择信息增益最大的特征作为该节点的特征，由该特征的不同取值建立子节点，在对子节点递归调用以上方法，构建决策树；直到所有特征的信息增益很小或者没有特征可以选择为止，最后得到一个决策树。

&emsp;&emsp;c4.5与ID3类似，唯一不同的是根据信息增益比选择特征。

In [25]:
# 定义节点类，是一个树
class Node:
    def __init__(self,root=True,label=None,feature_name=None,feature_axis=None):
        # root为True表示不能在往下递归调用了，已经是叶子节点了
        self.root = root
        self.label = label
        # 选择的特征名称
        self.feature_name = feature_name
        # 特征的axis坐标
        self.feature_axis = feature_axis
        # 树用字典表示，key是特征，value是子树
        self.tree = {}
        self.result = {
            'label':self.label,
            'feature_axis':self.feature_axis,
            'tree':self.tree
        }
        
    def __repr__(self):
        return '{}'.format(self.result)
    
    # node是特征值为val的子树，加入到该树中
    def add_node(self,val,node):
        self.tree[val] = node
    
    # features 是一个列表
    def predict(self,features):
        if self.root is True:
            return self.label
        return self.tree[features[self.feature_axis]].predict(features)
    
class DTree:
    def __init__(self,epsilon=0.1):
        self.epsilon = epsilon
        self.tree = {}
        
    # 熵
    @staticmethod
    def calc_ent(datasets):
        data_length = len(datasets)
        label_count = dict()
        for i in range(data_length):
            label = datasets[i][-1]
            label_count[label] = label_count.get(label,0) + 1
        ent = -sum([(p/data_length) * log(p/data_length,2) for p in label_count.values()])
        return ent
    
    
    # 计算条件熵
    @staticmethod
    def cond_ent(datasets,axis=0):
        # 计算条件熵肯定要借助于计算熵
        data_length = len(datasets)
        feature_sets = dict()
        for i in range(data_length):
            feature = datasets[i][axis]
            if feature not in feature_sets:
                feature_sets[feature] = []
            feature_sets[feature].append(datasets[i])
        cond_sum = sum([(len(p)/data_length) * calc_ent(p) for p in feature_sets.values()])
        return cond_sum 
    
    # 信息增益
    @staticmethod
    def info_gain(ent,cond_ent,label='minis'):
        if label == 'minis':
            return ent - cond_ent
        else:
            return cond_ent/ent
    
    # 选择最好的特征，返回(feature_axis,info_gain)
    def info_gain_train(self,datasets):
        # datasets 最后一列是类别
        feature_num = len(datasets[0]) - 1
        ent = calc_ent(datasets)

        best_feature = []
        for c in range(feature_num):
            c_info_gain = info_gain(ent ,cond_ent(datasets,axis=c))
            best_feature.append((c,c_info_gain))
            #print('特征({}) - info_gain - {:.3f}'.format(labels[c], c_info_gain))
        # 比较大小
        best_ = max(best_feature, key=lambda x: x[-1])
        return best_
    
    def train(self,train_data):
        '''
        input:数据集D(DataFrame格式)，特征集A,阈值eta
        output:决策树
        '''
        y_train,features = train_data.iloc[:,-1],train_data.columns[:-1]
        
        # 1.如果D中所有实例属于同一类Ck，则T为单节点树，并将Ck类作为该节点的类标记，返回T
        if len(y_train.value_counts()) == 1:
            return Node(root=True,label=y_train.iloc[0])
        
        # 2.如果A为空，则T为单节点树，将D中实例树最大的类Ck作为该节点的类标记，返回T
        if len(features) == 0:
            return Node(root=True,label=y_train.value_counts().sort_values(ascending=False).index[0])
        
        # 3.计算最大信息增益,Ag为信息增益最大的特征
        max_feature_index,max_info_gain = self.info_gain_train(np.array(train_data))
        max_feature_name = features[max_feature_index]
        
        # 4.如果Ag的信息增益小于阈值，则T为单节点，并将D中实例数最大的类Ck最为该节点的类标记
        if max_info_gain < self.epsilon:
            return Node(
                root=True,
                label = y_train.value_counts().sort_values(ascending=False).index[0]
            )
        
        # 5.构建Ag子集
        node_tree = Node(
            root=False,
            feature_name=max_feature_name,
            feature_axis=max_feature_index
        )
        
        # 接下来要做的是，统计feature的所有特征值，然后构建子树
        feature_list = train_data[max_feature_name].value_counts().index
        
        for f in feature_list:
            sub_train_df = train_data.loc[train_data[max_feature_name] == f].drop([max_feature_name],axis=1)
            
            # 6.递归的生成树
            sub_tree = self.train(sub_train_df)
            node_tree.add_node(f,sub_tree)
        return node_tree
    
    def fit(self,train_data):
        self._tree = self.train(train_data)
        return self._tree
    
    def predict(self,X_test):
        return self._tree.predict(X_test)
        

In [26]:
datasets, labels = create_data()
data_df = pd.DataFrame(datasets, columns=labels)
dt = DTree()
tree = dt.fit(data_df)

In [27]:
tree

{'label': None, 'feature_axis': 2, 'tree': {'否': {'label': None, 'feature_axis': 1, 'tree': {'否': {'label': '否', 'feature_axis': None, 'tree': {}}, '是': {'label': '是', 'feature_axis': None, 'tree': {}}}}, '是': {'label': '是', 'feature_axis': None, 'tree': {}}}}

In [28]:
dt.predict(['老年', '否', '否', '一般'])

'否'

In [32]:
data_df['类别'].unique()

array(['否', '是'], dtype=object)